In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from underthesea import sent_tokenize as underthesea_sent_tokenize # Renamed to avoid conflict
from summa.summarizer import summarize as summa_summarizer
import nltk
import re
import unicodedata
import os
import emoji # For preprocessing if needed

/data/elo/.conda/envs/vifree-txt-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

# Load the dataset
try:
    dataset = load_dataset("OpenHust/vietnamese-summarization")
    print("Dataset loaded successfully:")
    print(dataset)
except Exception as e:
    print(f"Error loading dataset: {e}")

# You can inspect the dataset structure, e.g., print the first example from the training set
# if 'train' in dataset:
#     print("\nExample from training set:")
#     print(dataset['train'][0])


Dataset loaded successfully:
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Document', 'Summary', 'Dataset'],
        num_rows: 74564
    })
})


In [3]:
dataset = dataset['train']

In [4]:
dataset

Dataset({
    features: ['Unnamed: 0', 'Document', 'Summary', 'Dataset'],
    num_rows: 74564
})

In [7]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:5


In [4]:
def text_rank_summarize(article, ratio):
    """
    Summarize the given article using the TextRank algorithm.

    Parameters:
    article (str): The text to be summarized.
    ratio (float): A number between 0 and 1 that determines the proportion of the number of sentences in the original text to be included in the summary.

    Returns:
    str: The summarized text.
    """
    # Summa's summarizer works best with text that has clear sentence boundaries.
    # It uses NLTK's sent_tokenize internally by default for English.
    # For Vietnamese, ensure sentences are well-separated (e.g., by periods).
    return summa_summarizer(article, ratio=ratio, language='english') # Summa might not have explicit Vietnamese support, relies on sentence splitting. NLTK's punkt can handle VI.

In [5]:
from summa.summarizer import summarize

example_long_text_vi = """
Hà Nội, thủ đô của Việt Nam, là một thành phố cổ kính với hơn một nghìn năm lịch sử. 
Nơi đây nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và một nền văn hóa ẩm thực đường phố phong phú. 
Du khách đến Hà Nội thường bị cuốn hút bởi sự pha trộn giữa nét cổ điển và hiện đại. 
Một trong những điểm đến không thể bỏ qua là Văn Miếu - Quốc Tử Giám, trường đại học đầu tiên của Việt Nam. 
Hồ Hoàn Kiếm, trái tim của thành phố, mang trong mình những truyền thuyết lịch sử và là nơi người dân địa phương tụ tập vui chơi. 
Ẩm thực Hà Nội cũng là một điểm nhấn đặc biệt, với các món ăn như phở, bún chả, và cà phê trứng đã trở thành biểu tượng. 
Tuy nhiên, thành phố cũng đối mặt với những thách thức như tắc nghẽn giao thông và ô nhiễm không khí. 
Chính quyền thành phố đang nỗ lực cải thiện cơ sở hạ tầng và nâng cao chất lượng cuộc sống cho người dân. 
Nhìn chung, Hà Nội vẫn là một điểm đến hấp dẫn với nhiều điều thú vị để khám phá.
"""
summary = summarize(example_long_text_vi, ratio=0.5)
print(summary)

Hà Nội, thủ đô của Việt Nam, là một thành phố cổ kính với hơn một nghìn năm lịch sử.
Nơi đây nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và một nền văn hóa ẩm thực đường phố phong phú.
Hồ Hoàn Kiếm, trái tim của thành phố, mang trong mình những truyền thuyết lịch sử và là nơi người dân địa phương tụ tập vui chơi.
Ẩm thực Hà Nội cũng là một điểm nhấn đặc biệt, với các món ăn như phở, bún chả, và cà phê trứng đã trở thành biểu tượng.


In [6]:
def run_extractive_summarizer(input_text, chosen_ratio):
    """
    Summarizes the input text using the TextRank algorithm, with a fallback to the first sentence if the summary is empty.

    Parameters:
    input_text (str or iterable): The text to be summarized. If it's not a string, it will be converted to one.
    chosen_ratio (float): A number between 0 and 1 that determines the proportion of the original text to be included in the summary. It's capped at a minimum of 0.05.

    Returns:
    str: The summary of the input text or the first sentence if the summary is empty.
    """
    if not isinstance(input_text, str):
        input_text = ' '.join(map(str, input_text))
    
    chosen_ratio = max(chosen_ratio, 0.05) # Ensure a minimum ratio

    summary = text_rank_summarize(input_text, ratio=chosen_ratio)
    
    # Fallback if summary is empty
    if summary:
        return summary
    else:
        # Using underthesea for more robust Vietnamese sentence tokenization for fallback
        sentences = underthesea_sent_tokenize(input_text)
        if sentences:
            return sentences[0]
        else:
            return "Unable to summarize the input text or extract the first sentence."

In [7]:
run_extractive_summarizer(example_long_text_vi,0.5)

'Hà Nội, thủ đô của Việt Nam, là một thành phố cổ kính với hơn một nghìn năm lịch sử.\nNơi đây nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và một nền văn hóa ẩm thực đường phố phong phú.\nHồ Hoàn Kiếm, trái tim của thành phố, mang trong mình những truyền thuyết lịch sử và là nơi người dân địa phương tụ tập vui chơi.\nẨm thực Hà Nội cũng là một điểm nhấn đặc biệt, với các món ăn như phở, bún chả, và cà phê trứng đã trở thành biểu tượng.'

In [7]:
sample_vietnamese_article_for_textrank = """
Hà Nội mùa thu, cây cơm nguội vàng, cây bàng lá đỏ, nằm kề bên nhau, phố xưa nhà cổ, mái ngói thâm nâu. 
Hồ Gươm, Tháp Rùa, nghiêng soi bóng, nước xanh như lọc. 
Những con đường ngoại ô thành phố vẫn giữ được vẻ đẹp bình dị với những cánh đồng lúa xanh mướt và những đàn cò trắng bay lượn. 
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống. 
Mỗi độ thu về, Hà Nội lại khoác lên mình một chiếc áo mới, lãng mạn và đầy chất thơ, níu chân biết bao du khách.
"""

print("--- TextRank Summarization Demo ---")
print(f"Original Article (length: {len(sample_vietnamese_article_for_textrank)} chars):\n{sample_vietnamese_article_for_textrank}")

summary_ratio = 0.4 # Summarize to 40% of original sentences
textrank_summary = run_extractive_summarizer(sample_vietnamese_article_for_textrank, summary_ratio)
print(f"\nTextRank Summary (ratio: {summary_ratio}, length: {len(textrank_summary)} chars):\n{textrank_summary}")

--- TextRank Summarization Demo ---
Original Article (length: 528 chars):

Hà Nội mùa thu, cây cơm nguội vàng, cây bàng lá đỏ, nằm kề bên nhau, phố xưa nhà cổ, mái ngói thâm nâu. 
Hồ Gươm, Tháp Rùa, nghiêng soi bóng, nước xanh như lọc. 
Những con đường ngoại ô thành phố vẫn giữ được vẻ đẹp bình dị với những cánh đồng lúa xanh mướt và những đàn cò trắng bay lượn. 
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống. 
Mỗi độ thu về, Hà Nội lại khoác lên mình một chiếc áo mới, lãng mạn và đầy chất thơ, níu chân biết bao du khách.


TextRank Summary (ratio: 0.4, length: 234 chars):
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống.
Mỗi độ thu về, Hà Nội lại khoác lên mình một chiếc áo mới, lãng mạn và đầy chất thơ, níu chân biết bao du khách.


In [9]:
MODEL_NAME_SUMM = "VietAI/vit5-base-vietnews-summarization"

try:
    tokenizer_summ = AutoTokenizer.from_pretrained(MODEL_NAME_SUMM)
    model_summ = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_SUMM)
    model_summ.to(device)
    model_summ.eval()
    print(f"Successfully loaded model: {MODEL_NAME_SUMM}")
except Exception as e:
    print(f"Error loading summarization model {MODEL_NAME_SUMM}: {e}")
    tokenizer_summ = None
    model_summ = None

/data/elo/.conda/envs/vifree-txt-3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Successfully loaded model: VietAI/vit5-base-vietnews-summarization


In [29]:
doc = dataset[0]['Document']
doc

'Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn .Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm .Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo ra những hình ảnh phản cảm đối với khách tham quan du lịch , người nước ngoài đang sinh sống và làm việc tại Hà Nội , gây ảnh hưởng tới hình ảnh của một thủ đô " văn minh , hiện đại " .Trong thực tế , theo ghi nhận của Tuổi Trẻ tại phố Tam Trinh , ngay đoạn đầu cầu Mai Động ( quận Hoàng Mai ) , một đoạn phố dài với gần chục cửa hàng buôn bán thịt chó sống , nhà hàng phục vụ các món chế biến từ thịt chó vẫn hoạt động tấp nập nhiều năm nay .Chị Th . , một chủ sạp bán thịt chó sống ở đây , cho ha

In [31]:
sum = dataset[0]['Summary']
sum

'Các quận , huyện , thị xã tuyên truyền bằng nhiều hình thức để người dân từ bỏ thói quen ăn thịt chó , mèo nhằm tránh nguy cơ mắc bệnh truyền nhiễm ( bệnh dại , xoắn khuẩn , tả ... ) cũng như không gây ảnh hưởng tới hình ảnh thủ đô văn minh , hiện đại .'

In [30]:
doc_sum = abstractive_summarize_vietnamese(doc, model_summ, tokenizer_summ)
doc_sum

'Sau khi báo chí đưa tin nhiều về các ca nhiễm khuẩn cầu lợn gây chết hàng loạt, gần 100 nhà hàng thịt chó ở Hà Nội đã đóng cửa không rõ lý do, chỉ còn duy nhất một cửa hàng bán thịt chó duy nhất ở quận Cầu Giấy và quận Hà Đông - nơi trước đây từng được coi là " thánh địa " của du lịch - đã bày tỏ quan điểm chấp nhận việc bỏ thói quen này để tránh gây phản cảm với du khách nước ngoài.Ựcả người dân thủ đô đều lên tiếng phản đối.'

In [35]:
predoc = preprocess_text_for_abstractive_summarization(doc)
print(predoc)
pre_doc_sum = abstractive_summarize_vietnamese(predoc, model_summ, tokenizer_summ)
print(pre_doc_sum)


Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn . Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm . Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo ra những hình ảnh phản cảm đối với khách tham quan du lịch , người nước ngoài đang sinh sống và làm việc tại Hà Nội , gây ảnh hưởng tới hình ảnh của một thủ đô " văn minh , hiện đại " . Trong thực tế , theo ghi nhận của Tuổi Trẻ tại phố Tam Trinh , ngay đoạn đầu cầu Mai Động ( quận Hoàng Mai ) , một đoạn phố dài với gần chục cửa hàng buôn bán thịt chó sống , nhà hàng phục vụ các món chế biến từ thịt chó vẫn hoạt động tấp nập nhiều năm nay . Chị Th . , một chủ sạp bán thịt chó sống ở đây , cho

In [ ]:
# Evaluate metrics between reference summary (sum) and generated summary (doc_sum)
import evaluate

print("--- Evaluation Metrics ---")
print(f"Reference Summary: {sum}")
print(f"Generated Summary: {doc_sum}")
print()

# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_score = bleu_metric.compute(predictions=[doc_sum], references=[[sum]])
print(f"BLEU Score: {bleu_score['bleu']:.4f}")

# ROUGE Scores
rouge_metric = evaluate.load("rouge")
rouge_scores = rouge_metric.compute(predictions=[doc_sum], references=[sum])
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")
print(f"ROUGE-Lsum: {rouge_scores['rougeLsum']:.4f}")

# METEOR Score
meteor_metric = evaluate.load("meteor")
meteor_score = meteor_metric.compute(predictions=[doc_sum], references=[sum])
print(f"METEOR Score: {meteor_score['meteor']:.4f}")

# BERTScore
bertscore_metric = evaluate.load("bertscore")
bertscore_results = bertscore_metric.compute(predictions=[doc_sum], references=[sum], lang="vi")
print(f"BERTScore Precision: {bertscore_results['precision'][0]:.4f}")
print(f"BERTScore Recall: {bertscore_results['recall'][0]:.4f}")
print(f"BERTScore F1: {bertscore_results['f1'][0]:.4f}")


--- Evaluation Metrics ---
Reference Summary: Các quận , huyện , thị xã tuyên truyền bằng nhiều hình thức để người dân từ bỏ thói quen ăn thịt chó , mèo nhằm tránh nguy cơ mắc bệnh truyền nhiễm ( bệnh dại , xoắn khuẩn , tả ... ) cũng như không gây ảnh hưởng tới hình ảnh thủ đô văn minh , hiện đại .
Generated Summary: Sau khi báo chí đưa tin nhiều về các ca nhiễm khuẩn cầu lợn gây chết hàng loạt, gần 100 nhà hàng thịt chó ở Hà Nội đã đóng cửa không rõ lý do, chỉ còn duy nhất một cửa hàng bán thịt chó duy nhất ở quận Cầu Giấy và quận Hà Đông - nơi trước đây từng được coi là " thánh địa " của du lịch - đã bày tỏ quan điểm chấp nhận việc bỏ thói quen này để tránh gây phản cảm với du khách nước ngoài.Ựcả người dân thủ đô đều lên tiếng phản đối.



BLEU Score: 0.0000
ROUGE-1: 0.5903
ROUGE-2: 0.2311
ROUGE-L: 0.3084
ROUGE-Lsum: 0.3084


[nltk_data] Downloading package wordnet to /home/elo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/elo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/elo/nltk_data...


METEOR Score: 0.2540
BERTScore Precision: 0.6741
BERTScore Recall: 0.6934
BERTScore F1: 0.6836


In [36]:
# Evaluate metrics between reference summary (sum) and generated summary (doc_sum)
import evaluate

print("--- Evaluation Metrics ---")
print(f"Reference Summary: {sum}")
print(f"Generated Summary: {pre_doc_sum}")
print()

# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_score = bleu_metric.compute(predictions=[pre_doc_sum], references=[[sum]])
print(f"BLEU Score: {bleu_score['bleu']:.4f}")

# ROUGE Scores
rouge_metric = evaluate.load("rouge")
rouge_scores = rouge_metric.compute(predictions=[pre_doc_sum], references=[sum])
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")
print(f"ROUGE-Lsum: {rouge_scores['rougeLsum']:.4f}")

# METEOR Score
meteor_metric = evaluate.load("meteor")
meteor_score = meteor_metric.compute(predictions=[pre_doc_sum], references=[sum])
print(f"METEOR Score: {meteor_score['meteor']:.4f}")

# BERTScore
bertscore_metric = evaluate.load("bertscore")
bertscore_results = bertscore_metric.compute(predictions=[pre_doc_sum], references=[sum], lang="vi")
print(f"BERTScore Precision: {bertscore_results['precision'][0]:.4f}")
print(f"BERTScore Recall: {bertscore_results['recall'][0]:.4f}")
print(f"BERTScore F1: {bertscore_results['f1'][0]:.4f}")


--- Evaluation Metrics ---
Reference Summary: Các quận , huyện , thị xã tuyên truyền bằng nhiều hình thức để người dân từ bỏ thói quen ăn thịt chó , mèo nhằm tránh nguy cơ mắc bệnh truyền nhiễm ( bệnh dại , xoắn khuẩn , tả ... ) cũng như không gây ảnh hưởng tới hình ảnh thủ đô văn minh , hiện đại .
Generated Summary: Sau khi báo chí đưa tin nhiều về các ca nhiễm khuẩn cầu lợn gây chết hàng loạt, gần 100 nhà hàng thịt chó ở Hà Nội đã đóng cửa không rõ lý do, chỉ còn duy nhất một cửa hàng bán thịt chó duy nhất ở quận Cầu Giấy và quận Hà Đông - nơi trước đây từng được coi là " thánh địa " của du lịch - đã bày tỏ quan điểm chấp nhận việc bỏ thói quen này để tránh gây phản cảm với du khách nước ngoài.Ựcả người dân thủ đô đều lên tiếng phản đối.

BLEU Score: 0.0000
ROUGE-1: 0.5903
ROUGE-2: 0.2311
ROUGE-L: 0.3084
ROUGE-Lsum: 0.3084


[nltk_data] Downloading package wordnet to /home/elo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/elo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/elo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR Score: 0.2540
BERTScore Precision: 0.6741
BERTScore Recall: 0.6934
BERTScore F1: 0.6836


In [10]:
import re
import unicodedata
# Assuming 'dicchar', 'emoji_pattern', 'bang_nguyen_am', 'nguyen_am_to_ids'
# and 'chuan_hoa_dau_tu_tieng_viet' and its helpers are defined as in your SA script.
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # Flags
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
    "]+", flags=re.UNICODE)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

def chuan_hoa_dau_cau_tieng_viet(sentence):
    words = sentence.split()
    for index, word in enumerate(words):
        # Preserve surrounding punctuation by splitting and processing only the word part
        match = re.match(r'(^[\W_]*)([\wÀ-Ỹà-ỹ._]*[\wÀ-Ỹà-ỹ]+)([\W_]*$)', word)
        if match:
            prefix, core_word, suffix = match.groups()
            normalized_core_word = chuan_hoa_dau_tu_tieng_viet(core_word)
            words[index] = prefix + normalized_core_word + suffix
        else: # If word doesn't match (e.g. pure punctuation or malformed), try to normalize if it's a simple word
            words[index] = chuan_hoa_dau_tu_tieng_viet(word) 
    return " ".join(words)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: continue
        if x == 9: # u
            if index > 0 and chars[index - 1].lower() == 'q':
                chars[index] = 'u'; qu_or_gi = True
        elif x == 5: # i
            if index > 0 and chars[index - 1].lower() == 'g':
                chars[index] = 'i'; qu_or_gi = True
        if y != 0:
            dau_cau = y; chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1: # Fix: check qu_or_gi correctly
            nguyen_am_index.append(index)

    if not nguyen_am_index: return "".join(chars)

    # Determine which vowel to place the tone mark on
    idx_to_mark = nguyen_am_index[0] # Default to the first vowel in the group
    if len(nguyen_am_index) >= 2:
        # Priority for ê, ơ, ô
        priority_vowel_found = False
        for idx_candidate in nguyen_am_index:
            x_vowel, _ = nguyen_am_to_ids.get(chars[idx_candidate], (-1,-1))
            if x_vowel in [4, 7, 8]: # ê, ô, ơ
                idx_to_mark = idx_candidate
                priority_vowel_found = True
                break
        
        if not priority_vowel_found:
            # Rules for diphthongs/triphthongs (simplified from original logic)
            # If the vowel group is at the end of the word
            if nguyen_am_index[-1] == len(chars) -1:
                # If ends with i, u, y (closed vowels/semivowels), mark the vowel before it
                x_last_vowel, _ = nguyen_am_to_ids.get(chars[nguyen_am_index[-1]], (-1,-1))
                if x_last_vowel in [5, 9, 10, 11]: # i, u, ư, y
                     idx_to_mark = nguyen_am_index[-2] if len(nguyen_am_index) > 1 else nguyen_am_index[-1]
                else: # Otherwise, mark the first vowel of the group (e.g., 'oa', 'oe')
                    idx_to_mark = nguyen_am_index[0]
            else: # Vowel group is followed by consonants (e.g., 'uyen', 'oan')
                if len(nguyen_am_index) == 3: # Triphthongs like 'uye', 'oai' -> mark middle
                    idx_to_mark = nguyen_am_index[1]
                elif len(nguyen_am_index) == 2: # Diphthongs like 'uyê', 'oa' -> mark second
                    idx_to_mark = nguyen_am_index[1]
                # else (single vowel before consonant), default (first vowel) is fine

    # Apply the tone mark
    x_target_vowel, _ = nguyen_am_to_ids.get(chars[idx_to_mark], (-1,-1))
    if x_target_vowel != -1 and dau_cau != 0:
        chars[idx_to_mark] = bang_nguyen_am[x_target_vowel][dau_cau]
    return "".join(chars)

nguyen_am_to_ids = {}
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

def normalize_unicode_summ(text): # Specific to summarization
    return unicodedata.normalize('NFC', text)

def convert_unicode_legacy_summ(txt): # Specific to summarization
    # Ensure dicchar is accessible here, e.g., passed as arg or global
    # For simplicity, assuming it's global or accessible
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# (You'll need chuan_hoa_dau_cau_tieng_viet and its dependencies like
#  is_valid_vietnam_word, nguyen_am_to_ids, bang_nguyen_am from your SA script)

def preprocess_text_for_abstractive_summarization(text):
    """
    Simplified and adapted preprocessing for abstractive summarization (e.g., ViT5).
    """
    if not isinstance(text, str):
        text = str(text)

    # 1. Remove URLs, mentions, hashtags (optional, depends on dataset)
    processed_text = re.sub(r"http\S+|www\S+|@\S+|#\S+", "", text) # Keep original case

    # 2. Legacy Unicode conversion (e.g., Windows-1258 to Unicode)
    processed_text = convert_unicode_legacy_summ(processed_text)

    # 3. Standard Unicode Normalization (NFC)
    processed_text = normalize_unicode_summ(processed_text)

    # 4. Remove Emojis (replace with space)
    processed_text = re.sub(emoji_pattern, " ", processed_text)

    # 5. Reduce repeated alphabetic characters (handles Vietnamese characters)
    # Kept original case for this regex by removing flags=re.IGNORECASE or ensuring it handles case correctly
    processed_text = re.sub(r'([a-zA-Zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ])\1+', r'\1', processed_text)

    # 6. Reduce repeated special characters (non-alphanumeric, non-whitespace, non-Vietnamese)
    # This regex is case-insensitive by nature of [^a-z0-9...]
    processed_text = re.sub(r'([^a-zA-Z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s])\1+', r'\1', processed_text)

    # 7. Normalize punctuation spacing (carefully, preserving punctuation)
    _punctuation_chars_summ = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' # Consider if all are needed
    escaped_punctuation_summ = re.escape(_punctuation_chars_summ)
    # Add space around punctuation, but not if it's part of a number (e.g., 3.14)
    # This is a simplified version; robustly handling numbers.decimal vs sentence punctuation can be tricky.
    # For ViT5, often minimal intervention is better.
    # The primary goal is to separate stuck punctuation: "word.Another" -> "word . Another"
    processed_text = re.sub(r'(?<=[^\W\d_])([' + escaped_punctuation_summ + r'])', r' \1', processed_text) # Add space before if preceded by word char
    processed_text = re.sub(r'([' + escaped_punctuation_summ + r'])(?=[^\W\d_])', r'\1 ', processed_text) # Add space after if followed by word char


    # 8. Reduce repeated punctuation characters (e.g., "!!!" to "!")
    processed_text = re.sub(r"([" + escaped_punctuation_summ + r"])\1+", r"\1", processed_text)

    # 9. Vietnamese tone mark normalization
    processed_text = chuan_hoa_dau_cau_tieng_viet(processed_text) # This function should handle case appropriately

    # 10. Final whitespace cleanup (multiple spaces to single, strip ends)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()

    return processed_text

In [12]:
# def normalize_unicode_summ(text):
#     return unicodedata.normalize('NFC', text)

# def preprocess_text_for_abstractive_summarization(text):
#     text = str(text)
#     text = normalize_unicode_summ(text)
#     # Minimal cleaning for abstractive models; they often handle noise well.
#     # Remove excessive newlines or tabs, but preserve sentence structure.
#     text = re.sub(r'\\n+', '\\n', text) # Consolidate multiple newlines
#     text = re.sub(r'[\\t ]+', ' ', text) # Consolidate multiple spaces/tabs to single space
#     text = text.strip()
#     return text

In [11]:
def abstractive_summarize_vietnamese(document_text, model, tokenizer, 
                                     target_ratio = None,
                                     max_input_length=1024, # Max length for T5 base input
                                     min_summary_length=100, 
                                     max_summary_length=500, 
                                     num_beams=4, 
                                     length_penalty=1.2, 
                                     no_repeat_ngram_size=3, 
                                     early_stopping=True):
    if not model or not tokenizer:
        return "Error: Abstractive summarization model or tokenizer not loaded."

    document_text = preprocess_text_for_abstractive_summarization(document_text)
    min_len_to_use = min_summary_length
    max_len_to_use = max_summary_length

    if target_ratio is not None and 0 < target_ratio <= 1.0:
        try:
            sentences_in_doc = underthesea_sent_tokenize(document_text)
            num_sentences_in_doc = len(sentences_in_doc)

            if num_sentences_in_doc > 0:
                target_num_sentences = max(1, int(num_sentences_in_doc * target_ratio))
                
                # Heuristic: Average tokens per Vietnamese sentence for news-style text.
                # This is an estimate and might need adjustment based on your specific domain.
                # Common Vietnamese sentence length: 15-25 words.
                # Tokens per word for Vietnamese BERT-style tokenizers can be 1 to 1.5+
                # Let's estimate ~20-30 tokens per sentence.
                avg_tokens_per_sentence_heuristic = 25 
                
                ideal_summary_tokens = target_num_sentences * avg_tokens_per_sentence_heuristic
                
                # Set min/max summary lengths based on this ideal token count
                # Ensure min_len is at least a small absolute number (e.g., 15-20 tokens)
                # Ensure max_len is also reasonable (e.g., not exceeding ~250-300 for typical summaries)
                min_len_to_use = max(100, int(ideal_summary_tokens * 0.6)) 
                max_len_to_use = min(500, int(ideal_summary_tokens * 1.4) + 10) # Add a small buffer 

                # Ensure min_length < max_length and handle edge cases
                if min_len_to_use >= max_len_to_use:
                    min_len_to_use = max(15, max_len_to_use // 2)
                if max_len_to_use <= min_len_to_use : # if max_len_to_use became too small
                    max_len_to_use = min_len_to_use + 15 # Ensure there's some room

        except Exception as e:
            print(f"  Ratio mode: Error calculating ratio-based lengths ({e}), using default summary lengths.")

    input_text = document_text

    encoding = tokenizer(input_text, return_tensors="pt", max_length=max_input_length, truncation=True, padding="longest").to(device)

    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask

    summary_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_len_to_use,
        min_length=min_len_to_use,
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        early_stopping=early_stopping
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return summary

In [21]:
def abstractive_summarize_vietnamese(documents_texts, model, tokenizer, 
                                     target_ratio = None,
                                     max_input_length=1024, # Max length for T5 base input
                                     min_summary_length=100, 
                                     max_summary_length=500, 
                                     num_beams=4, 
                                     length_penalty=1.2, 
                                     no_repeat_ngram_size=3, 
                                     early_stopping=True):
    if not model or not tokenizer:
        error_message = "Error: Abstractive summarization model or tokenizer not loaded."
        if isinstance(documents_texts, list):
            return [error_message] * len(documents_texts)
        else:
            return error_message

    is_batch = isinstance(documents_texts, list)
    if not is_batch:
        # If a single string is passed, wrap it in a list for uniform processing
        documents_texts = [documents_texts] 
    
    # Preprocess each document text individually
    # Ensure preprocess_text_for_abstractive_summarization is defined correctly elsewhere
    processed_texts = [preprocess_text_for_abstractive_summarization(doc) for doc in documents_texts]    

    min_len_final = min_summary_length
    max_len_final = max_summary_length

    # Target ratio logic: Apply only if it's a single document input (not a batch)
    # and target_ratio is provided. For batch, fixed min/max will be used.
    if not is_batch and target_ratio is not None and 0 < target_ratio <= 1.0:
        doc_for_ratio_calc = processed_texts[0] # The single preprocessed document
        try:
            # Ensure underthesea_sent_tokenize is imported and available
            sentences_in_doc = underthesea_sent_tokenize(doc_for_ratio_calc) 
            num_sentences_in_doc = len(sentences_in_doc)

            if num_sentences_in_doc > 0:
                target_num_sentences = max(1, int(num_sentences_in_doc * target_ratio))
                avg_tokens_per_sentence_heuristic = 25 
                ideal_summary_tokens = target_num_sentences * avg_tokens_per_sentence_heuristic
                
                min_len_final = max(100, int(ideal_summary_tokens * 0.6)) 
                max_len_final = min(500, int(ideal_summary_tokens * 1.4) + 10)

                if min_len_final >= max_len_final:
                    min_len_final = max(15, max_len_final // 2) 
                if max_len_final <= min_len_final: 
                    max_len_final = min_len_final + 15 
                
                # Optional: for debugging
                # print(f"  Ratio mode (single doc): Input sentences: {num_sentences_in_doc}, Target sentences (est.): {target_num_sentences}")
                # print(f"  Ratio mode (single doc): Est. ideal tokens: {ideal_summary_tokens}, Using min_length: {min_len_final}, max_length: {max_len_final}")
            # else:
                # print("  Ratio mode (single doc): No sentences found in input, using default summary lengths.")
        except Exception as e:
            # print(f"  Ratio mode (single doc): Error calculating ratio-based lengths ({e}), using default summary lengths.")
            # Fallback to the global min_summary_length and max_summary_length if ratio calc fails
            min_len_final = min_summary_length 
            max_len_final = max_summary_length
    # else:
        # Optional: for debugging
        # print(f"  Ratio not provided or invalid, or processing a batch. Using default summary lengths: min={min_len_final}, max={max_len_final}")

    input_texts_for_tokenizer = processed_texts # List of preprocessed texts

    encoding = tokenizer(
        input_texts_for_tokenizer, 
        return_tensors="pt", 
        max_length=max_input_length, 
        truncation=True, 
        padding="longest"
    ).to(device)

    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask

    summary_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_len_final, # Use the determined max length
        min_length=min_len_final, # Use the determined min length
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        early_stopping=early_stopping
    )

    # Decode all summaries in the batch
    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # If the original input was a single string, return a single string for backward compatibility.
    if not is_batch:
        return summaries[0] if summaries else ""
    
    return summaries # Return the list of summaries for batch input

In [ ]:
# Test batch generation with abstractive_summarize_vietnamese

# Define a batch of sample Vietnamese documents
# Document 1 from notebook_cell_output_7
doc1_batch_test = 'Hà Nội, thủ đô của Việt Nam, là một thành phố cổ kính với hơn một nghìn năm lịch sử.\nNơi đây nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và một nền văn hóa ẩm thực đường phố phong phú.\nHồ Hoàn Kiếm, trái tim của thành phố, mang trong mình những truyền thuyết lịch sử và là nơi người dân địa phương tụ tập vui chơi.\nẨm thực Hà Nội cũng là một điểm nhấn đặc biệt, với các món ăn như phở, bún chả, và cà phê trứng đã trở thành biểu tượng.'

# Document 2 based on notebook_cell_output_5 (partial content)
doc2_batch_test = 'Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn .Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm .Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo những hình ảnh không đẹp , phản cảm đối với du khách trong và ngoài nước , làm ảnh hưởng đến hình ảnh của một thủ đô văn minh , hiện đại .'

# Document 3: A generic short text
doc3_batch_test = "Việt Nam là một quốc gia có tiềm năng lớn về du lịch biển với đường bờ biển dài và nhiều bãi biển đẹp. Phát triển du lịch bền vững gắn liền với bảo vệ môi trường biển là một ưu tiên hàng đầu."

sample_documents_batch = [doc1_batch_test, doc2_batch_test, doc3_batch_test]

print("--- Testing abstractive_summarize_vietnamese with batch input ---")
print(f"Number of documents in batch: {len(sample_documents_batch)}")

# Ensure model, tokenizer, device are loaded and available in the global scope
# from previous cells.
# Also, abstractive_summarize_vietnamese and its helper 
# preprocess_text_for_abstractive_summarization must be defined.

# Test 1: Batch summarization with specified min/max lengths
print("\n--- Test 1: Batch summarization with fixed min/max lengths (20/100) ---")
try:
    batch_summaries_fixed_lengths = abstractive_summarize_vietnamese(
        documents_texts=sample_documents_batch,
        model=model_summ, 
        tokenizer=tokenizer_summ, 
        max_input_length=1024,  # Default from function
        min_summary_length=100,  # Custom min length for this test
        max_summary_length=500, # Custom max length for this test
        num_beams=4,            # Common value, can be function's default (5)
        length_penalty=1.0,     # Function's default
        no_repeat_ngram_size=3, # Function's default
        early_stopping=True     # Function's default
    )

    if isinstance(batch_summaries_fixed_lengths, list):
        for i, summary in enumerate(batch_summaries_fixed_lengths):
            print(f"\nSummary for Document {i+1}:")
            # print(f"Original (first 100 chars): {sample_documents_batch[i][:100].replace(chr(10), ' ')}...")
            print(f"Generated Summary: {summary}")
    else:
        print(f"ERROR: Expected a list of summaries, but got: {type(batch_summaries_fixed_lengths)}")
except Exception as e:
    print(f"ERROR during Test 1 (fixed lengths): {e}")

# Test 2: Batch summarization with summary_ratio
# Note: The current implementation of abstractive_summarize_vietnamese, based on the provided snippets,
# is expected to use the global min_summary_length and max_summary_length for batch processing,
# even if summary_ratio is provided, because model.generate() needs fixed lengths for a batch.
print("\n--- Test 2: Batch summarization with summary_ratio=0.25 ---")
print("(Expecting lengths to be governed by min_summary_length=30, max_summary_length=200 from function defaults if ratio doesn't override for batch)")
try:
    batch_summaries_ratio_test = abstractive_summarize_vietnamese(
        documents_texts=sample_documents_batch,
        model=model_summ,
        tokenizer=tokenizer_summ,
        # Using function's default min/max lengths which would be used if ratio is ignored for batch
        target_ratio=0.5,
        min_summary_length=100, 
        max_summary_length=500,
        # Other parameters can be defaults or specified
        num_beams=4,
        length_penalty=1.0,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    if isinstance(batch_summaries_ratio_test, list):
        for i, summary in enumerate(batch_summaries_ratio_test):
            print(f"\nSummary for Document {i+1} (ratio 0.25 attempt):")
            # print(f"Original (first 100 chars): {sample_documents_batch[i][:100].replace(chr(10), ' ')}...")
            print(f"Generated Summary: {summary}")
            print(f"Word count: {len(summary.split())}") # To observe actual length
    else:
        print(f"ERROR: Expected a list of summaries for batch input with ratio, but got: {type(batch_summaries_ratio_test)}")
except Exception as e:
    print(f"ERROR during Test 2 (summary_ratio): {e}")

# print("\n--- Batch summarization test finished ---")
# print("Note: The behavior of `summary_ratio` in batch mode depends on the `abstractive_summarize_vietnamese` function's implementation.")
# print("If it calculates a single min/max length for the entire batch (e.g., based on average or first item), results will reflect that.")
# print("If it falls back to global min/max lengths for batches, that will be observed.")


--- Testing abstractive_summarize_vietnamese with batch input ---
Number of documents in batch: 3

--- Test 1: Batch summarization with fixed min/max lengths (20/100) ---

Summary for Document 1:
Generated Summary: Các món ăn đường phố như phở, bún chả, và cà phê trứng đã trở thành biểu tượng. Trong đó, Hà Nội nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và nền văn hoá ẩm thực đường phố mang đậm văn hoá vùng kinh tế châu Á - Thái Bình Dương cũng mang trong mình một nét đặc trưng trong văn hoá giao thông đặc sắc. Hãy cùng ngắm nhìn thành phố từ những góc phố, con phố và cuộc sống nơi đây qua bài viết sau.

Summary for Document 2:
Generated Summary: Việc kinh doanh, giết mổ và sử dụng thịt chó, mèo ở Hà Nội thời gian qua đã gây những hình ảnh không đẹp, phản cảm đối du khách trong và ngoài nước. Hà Nội cũng đã và đang có nhiều biện pháp để chấn chỉnh tình trạng này. Hãy cùng nhìn lại thủ đô văn minh, hiện đại của Hà Nội. Tại sao thịt chó lại có hình ảnh phản cảm đến du khác

In [19]:
print(abstractive_summarize_vietnamese(doc1_batch_test, model_summ, tokenizer_summ, target_ratio=0.5))

Các món ăn đường phố như phở, bún chả, và cà phê trứng đã trở thành biểu tượng. Trong đó, Hà Nội nổi tiếng với kiến trúc Pháp thuộc, những hồ nước yên bình và nền văn hoá ẩm thực đường phố phong phú.


In [ ]:
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
import evaluate
from tqdm.auto import tqdm # Ensure this is imported for progress bars
import time
import gc
import numpy as np
import json
from datetime import datetime
import re # Make sure re is imported for preprocessing functions
import unicodedata # Make sure unicodedata is imported for preprocessing

# --- Ensure all necessary helper functions for preprocessing are defined above this cell ---
# This includes:
# - emoji_pattern, bang_nguyen_am, nguyen_am_to_ids, dicchar, loaddicchar (from cell execution_count: 11)
# - normalize_unicode_nfc_summ, convert_unicode_legacy_summ (from cell execution_count: 11)
# - is_valid_vietnam_word, chuan_hoa_dau_tu_tieng_viet (these are complex, ensure they are defined correctly if used by preprocess_text_for_abstractive_summarization)
# - preprocess_text_for_abstractive_summarization (from cell execution_count: 11)
# - abstractive_summarize_vietnamese (from cell execution_count: 24)
# - underthesea_sent_tokenize (if used by abstractive_summarize_vietnamese for ratio calculation)

# --- Cell for Dataset Splitting, Preprocessing References, and Evaluation ---

print("STEP 1: Loading and splitting the dataset...")
try:
    # Load the full dataset
    full_dataset = load_dataset("OpenHust/vietnamese-summarization")
    if 'train' not in full_dataset:
        raise ValueError("'train' split not found in the loaded dataset.")

    # Convert the 'train' split to a pandas DataFrame for splitting
    df = full_dataset['train'].to_pandas()

    # Split 80% for training, 20% for validation
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    print(f"Dataset split complete:")
    print(f"  Training set size: {len(train_df)}")
    print(f"  Validation set size: {len(val_df)}")

    # Convert pandas DataFrames back to Hugging Face Dataset objects
    train_dataset_hf = Dataset.from_pandas(train_df)
    val_dataset_hf = Dataset.from_pandas(val_df)

    # Create a new DatasetDict
    split_dataset = DatasetDict({
        'train': train_dataset_hf,
        'validation': val_dataset_hf
    })
    current_val_dataset = split_dataset['validation'] # Use this for evaluation

except Exception as e:
    print(f"Error during dataset loading/splitting: {e}")
    # Fallback or stop execution if dataset is crucial
    current_val_dataset = None


if current_val_dataset:
    print("\nSTEP 2: Preparing validation data (preprocessing reference summaries)...")
    # We will feed raw documents to `abstractive_summarize_vietnamese` as it preprocesses internally.
    # We need to preprocess the reference summaries for a fair comparison with the output.
    raw_val_docs = []
    preprocessed_val_references = []

    for example in tqdm(current_val_dataset, desc="Preprocessing references"):
        doc = example.get('Document', '')
        summary = example.get('Summary', '')

        if doc and summary: # Ensure both document and summary exist
            raw_val_docs.append(doc)
            # Preprocess the reference summary using the same function that
            # abstractive_summarize_vietnamese uses for its input.
            preprocessed_val_references.append(preprocess_text_for_abstractive_summarization(summary))
        elif doc: # If only document exists, still keep it for generation if desired (though no reference for it)
            raw_val_docs.append(doc)
            preprocessed_val_references.append("") # No reference to compare against

    print(f"  Validation set prepared: {len(raw_val_docs)} documents, {len(preprocessed_val_references)} preprocessed reference summaries.")

    # Filter out pairs where the preprocessed reference became empty, if that's undesirable
    valid_indices_for_eval = [
        i for i, ref in enumerate(preprocessed_val_references) if ref.strip()
    ]
    
    eval_docs = [raw_val_docs[i] for i in valid_indices_for_eval]
    eval_references = [preprocessed_val_references[i] for i in valid_indices_for_eval]

    if not eval_docs:
        print("Warning: No valid document-reference pairs after preprocessing references. Evaluation cannot proceed.")
    else:
        print(f"  Proceeding with {len(eval_docs)} valid pairs for evaluation.")

        print("\nSTEP 3: Loading evaluation metrics...")
        rouge_metric = evaluate.load('rouge')
        try:
            bertscore_metric = evaluate.load('bertscore')
            use_bertscore = True
            print("  BERTScore metric loaded successfully.")
        except Exception as e:
            print(f"  Warning: BERTScore metric not loaded: {e}. Proceeding with ROUGE only.")
            use_bertscore = False

        print("\nSTEP 4: Evaluating pre-trained model using `abstractive_summarize_vietnamese` function...")
        # Ensure model_summ, tokenizer_summ, and device are defined and loaded from previous cells
        if 'model_summ' not in locals() or 'tokenizer_summ' not in locals() or 'device' not in locals():
            print("Error: `model_summ`, `tokenizer_summ`, or `device` is not defined. Please run the model loading cell.")
        else:
            model_summ.eval() # Ensure model is in evaluation mode
            generated_summaries_for_eval = []
            inference_batch_size = 32
            start_time_eval = time.time()

            # Consider batching if `abstractive_summarize_vietnamese` processes one by one and it's slow
            # For now, processing one by one as per typical use of such a helper function
            # for i, doc_text in enumerate(tqdm(eval_docs, desc="Generating summaries")):
            for i in tqdm(range(0,len(eval_docs),inference_batch_size), desc = "Generating summaries as batch"):
                try:
                    # Call your existing function to generate the summary
                    # It uses its own internal preprocessing for the document text.
                    batch_document_texts = eval_docs[i:i+inference_batch_size]
                    batch_generated_summaries = abstractive_summarize_vietnamese(
                        batch_document_texts,
                        model_summ,
                        tokenizer_summ,
                        max_input_length=1024,  # Default from function
                        min_summary_length=100,  # Custom min length for this test
                        max_summary_length=500, # Custom max length for this test
                        num_beams=4,            # Common value, can be function's default (5)
                        length_penalty=1.0,     # Function's default
                        no_repeat_ngram_size=3, # Function's default
                        early_stopping=True     # Function's default
                    )
                    generated_summaries_for_eval.extend(batch_generated_summaries)
                except Exception as e:
                    print(f"Error generating summary for document index {i}: {e}")
                    # generated_summaries_for_eval.append("") # Append empty string on error
                    generated_summaries_for_eval.extend([""] * len(batch_document_texts)) # Add empty strings for failed batch


                if i > 0 and i % 50 == 0 : # Optional: Clear cache periodically if memory is an issue
                     if device.type == 'cuda': torch.cuda.empty_cache(); gc.collect()


            end_time_eval = time.time()
            print(f"  Summary generation complete. Time taken: {end_time_eval - start_time_eval:.2f} seconds.")

            print("\nSTEP 5: Computing metrics...")
            try:
                rouge_scores = rouge_metric.compute(
                    predictions=generated_summaries_for_eval,
                    references=eval_references,
                    use_stemmer=True
                )
                print("\n--- ROUGE Scores (Pre-trained Model) ---")
                for metric, score in rouge_scores.items(): print(f"  {metric}: {score*100:.2f}%")
            except Exception as e: print(f"Error computing ROUGE scores: {e}")

            if use_bertscore:
                try:
                    bertscore_results = bertscore_metric.compute(
                        predictions=generated_summaries_for_eval,
                        references=eval_references,
                        lang='vi', model_type="vinai/phobert-base", device=device,
                        batch_size=32, verbose=False # Bertscore also has batch_size
                    )
                    print("\n--- BERTScore (Pre-trained Model) ---")
                    avg_precision = np.mean(bertscore_results['precision']) if bertscore_results['precision'] else 0
                    avg_recall = np.mean(bertscore_results['recall']) if bertscore_results['recall'] else 0
                    avg_f1 = np.mean(bertscore_results['f1']) if bertscore_results['f1'] else 0
                    print(f"  Average Precision: {avg_precision*100:.2f}%")
                    print(f"  Average Recall: {avg_recall*100:.2f}%")
                    print(f"  Average F1-score: {avg_f1*100:.2f}%")
                except Exception as e:
                    print(f"Error computing BERTScore: {e}")
                    print("  BERTScore might require `pip install bert_score` and internet access.")

            print("\nSTEP 6: Displaying a few examples...")
            for i in range(min(3, len(generated_summaries_for_eval))):
                print(f"\n--- Example {i+1} ---")
                print(f"  Original Document (first 300 chars): {eval_docs[i][:300]}...")
                print(f"  Preprocessed Reference Summary: {eval_references[i]}")
                print(f"  Generated Summary (ViT5 Pre-trained): {generated_summaries_for_eval[i]}")

            print("\nSTEP 7: Saving evaluation results...")
            # (Saving logic remains the same as your previous cell)
            results_payload = {
                'timestamp': datetime.now().isoformat(),
                'model_name': MODEL_NAME_SUMM if 'MODEL_NAME_SUMM' in locals() else "N/A",
                'num_validation_samples_evaluated': len(eval_docs),
                'inference_batch_size': inference_batch_size,
                'evaluation_time_seconds': end_time_eval - start_time_eval,
                'rouge_scores': rouge_scores if 'rouge_scores' in locals() else None,
                'example_outputs': [
                    {
                        'document_preview': eval_docs[j][:300]+"...",
                        'reference_summary': eval_references[j],
                        'generated_summary': generated_summaries_for_eval[j]
                    } for j in range(min(10, len(eval_docs)))
                ]
            }
            if use_bertscore and 'bertscore_results' in locals() and bertscore_results:
                results_payload['bertscore'] = {
                    'precision_avg': np.mean(bertscore_results['precision']),
                    'recall_avg': np.mean(bertscore_results['recall']),
                    'f1_avg': np.mean(bertscore_results['f1'])
                }
            results_filename = f"evaluation_results_batched_ViT5_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            try:
                with open(results_filename, 'w', encoding='utf-8') as f:
                    json.dump(results_payload, f, ensure_ascii=False, indent=4)
                print(f"  Evaluation results saved to: {results_filename}")
            except Exception as e: print(f"Error saving results to JSON: {e}")
else:
    print("Evaluation cannot proceed as `current_val_dataset` is not available.")
print("\n--- Batched Evaluation Cell Execution Finished ---")

STEP 1: Loading and splitting the dataset...
Dataset split complete:
  Training set size: 59651
  Validation set size: 14913

STEP 2: Preparing validation data (preprocessing reference summaries)...


Preprocessing references: 100%|██████████| 14913/14913 [00:05<00:00, 2761.07it/s]


  Validation set prepared: 14913 documents, 14913 preprocessed reference summaries.
  Proceeding with 14913 valid pairs for evaluation.

STEP 3: Loading evaluation metrics...
  BERTScore metric loaded successfully.

STEP 4: Evaluating pre-trained model using `abstractive_summarize_vietnamese` function...


Generating summaries as batch:   0%|          | 1/467 [00:03<28:38,  3.69s/it]

Error generating summary for document index 0: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 5 has a total capacty of 79.15 GiB of which 239.88 MiB is free. Process 1611699 has 1.34 GiB memory in use. Process 2886948 has 1.56 GiB memory in use. Process 3013142 has 1.56 GiB memory in use. Process 981290 has 27.21 GiB memory in use. Process 2518409 has 20.19 GiB memory in use. Including non-PyTorch memory, this process has 27.02 GiB memory in use. Of the allocated memory 26.12 GiB is allocated by PyTorch, and 421.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


...

Generating summaries as batch:   7%|▋         | 35/467 [01:15<14:30,  2.01s/it]

Error generating summary for document index 1088: CUDA out of memory. Tried to allocate 356.00 MiB. GPU 5 has a total capacty of 79.15 GiB of which 239.88 MiB is free. Process 1611699 has 1.34 GiB memory in use. Process 2886948 has 1.56 GiB memory in use. Process 3013142 has 1.56 GiB memory in use. Process 981290 has 27.21 GiB memory in use. Process 2518409 has 20.19 GiB memory in use. Including non-PyTorch memory, this process has 27.02 GiB memory in use. Of the allocated memory 25.66 GiB is allocated by PyTorch, and 890.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


Generating summaries as batch:   7%|▋         | 35/467 [01:18<16:08,  2.24s/it]


KeyboardInterrupt: 

: 

In [22]:
sample_vietnamese_article_for_abstractive = """
Đại dịch COVID-19 đã gây ra những tác động sâu rộng đến kinh tế toàn cầu trong suốt hơn hai năm qua. 
Nhiều ngành công nghiệp chủ chốt như du lịch, hàng không, và dịch vụ ăn uống đã phải đối mặt với những thách thức chưa từng có. 
Các chính phủ trên khắp thế giới đã tung ra các gói kích thích kinh tế khổng lồ để hỗ trợ doanh nghiệp và người lao động. 
Tuy nhiên, quá trình phục hồi được dự báo sẽ không đồng đều và còn nhiều bất ổn phía trước, đặc biệt là với sự xuất hiện của các biến chủng mới. 
Chuyển đổi số và ứng dụng công nghệ được coi là chìa khóa quan trọng giúp các nền kinh tế thích ứng và vượt qua khủng hoảng. 
Việc đảm bảo chuỗi cung ứng toàn cầu và tăng cường hợp tác quốc tế cũng là những yếu tố then chốt cho sự phục hồi bền vững.
"""

print("--- Abstractive Summarization Demo (VietAI/vit5-base-vietnews-summarization) ---")
print(f"Original Article (length: {len(sample_vietnamese_article_for_abstractive)} chars):\n{sample_vietnamese_article_for_abstractive}")

if model_summ and tokenizer_summ:
    abstractive_summary = abstractive_summarize_vietnamese(sample_vietnamese_article_for_abstractive, model_summ, tokenizer_summ)
    print(f"\nAbstractive Summary (length: {len(abstractive_summary)} chars):\n{abstractive_summary}")
else:
    print("\nAbstractive summarization model not loaded. Skipping demo.")

--- Abstractive Summarization Demo (VietAI/vit5-base-vietnews-summarization) ---
Original Article (length: 751 chars):

Đại dịch COVID-19 đã gây ra những tác động sâu rộng đến kinh tế toàn cầu trong suốt hơn hai năm qua. 
Nhiều ngành công nghiệp chủ chốt như du lịch, hàng không, và dịch vụ ăn uống đã phải đối mặt với những thách thức chưa từng có. 
Các chính phủ trên khắp thế giới đã tung ra các gói kích thích kinh tế khổng lồ để hỗ trợ doanh nghiệp và người lao động. 
Tuy nhiên, quá trình phục hồi được dự báo sẽ không đồng đều và còn nhiều bất ổn phía trước, đặc biệt là với sự xuất hiện của các biến chủng mới. 
Chuyển đổi số và ứng dụng công nghệ được coi là chìa khóa quan trọng giúp các nền kinh tế thích ứng và vượt qua khủng hoảng. 
Việc đảm bảo chuỗi cung ứng toàn cầu và tăng cường hợp tác quốc tế cũng là những yếu tố then chốt cho sự phục hồi bền vững.

  Ratio not provided or invalid, using default summary lengths: min=100, max=500

Abstractive Summary (length: 427 chars):
Sự phá

In [51]:
longer_vietnamese_text = """
Thị trường bất động sản Việt Nam trong những năm gần đây đã trải qua nhiều biến động đáng chú ý. 
Sau giai đoạn tăng trưởng nóng, thị trường đã có dấu hiệu chững lại ở một số phân khúc, đặc biệt là đất nền và bất động sản nghỉ dưỡng ở các tỉnh lẻ. 
Nguyên nhân chính được cho là do các chính sách siết chặt tín dụng vào bất động sản, cũng như những biến động của kinh tế vĩ mô và tâm lý thận trọng của nhà đầu tư. 
Tuy nhiên, phân khúc nhà ở thực, đặc biệt là căn hộ chung cư tại các thành phố lớn như Hà Nội và TP.HCM, vẫn duy trì được sức hút nhất định do nhu cầu ở thực tế cao. 
Các chuyên gia dự báo rằng thị trường sẽ cần thêm thời gian để điều chỉnh và tìm lại điểm cân bằng mới. 
Trong dài hạn, với dân số trẻ và tốc độ đô thị hóa nhanh, tiềm năng phát triển của thị trường bất động sản Việt Nam vẫn được đánh giá cao. 
Các yếu tố như cải thiện cơ sở hạ tầng, sự minh bạch trong quy hoạch và chính sách pháp lý ổn định sẽ đóng vai trò quan trọng trong việc thúc đẩy sự phát triển bền vững của thị trường.
"""

print("--- Summarization Comparison --- ")
print(f"Original Text (length: {len(longer_vietnamese_text)} chars):\n{longer_vietnamese_text}")

# TextRank
textrank_comp_summary = run_extractive_summarizer(longer_vietnamese_text, chosen_ratio=0.3)
print(f"\nTextRank Summary (ratio 0.3, length: {len(textrank_comp_summary)} chars):\n{textrank_comp_summary}")

# Abstractive (VietAI T5)
if model_summ and tokenizer_summ:
    abstractive_comp_summary = abstractive_summarize_vietnamese(longer_vietnamese_text, model_summ, tokenizer_summ)
    print(f"\nAbstractive Summary (VietAI T5, length: {len(abstractive_comp_summary)} chars):\n{abstractive_comp_summary}")
else:
    print("\nAbstractive summarization model not loaded. Skipping comparison for it.")

--- Summarization Comparison --- 
Original Text (length: 1013 chars):

Thị trường bất động sản Việt Nam trong những năm gần đây đã trải qua nhiều biến động đáng chú ý. 
Sau giai đoạn tăng trưởng nóng, thị trường đã có dấu hiệu chững lại ở một số phân khúc, đặc biệt là đất nền và bất động sản nghỉ dưỡng ở các tỉnh lẻ. 
Nguyên nhân chính được cho là do các chính sách siết chặt tín dụng vào bất động sản, cũng như những biến động của kinh tế vĩ mô và tâm lý thận trọng của nhà đầu tư. 
Tuy nhiên, phân khúc nhà ở thực, đặc biệt là căn hộ chung cư tại các thành phố lớn như Hà Nội và TP.HCM, vẫn duy trì được sức hút nhất định do nhu cầu ở thực tế cao. 
Các chuyên gia dự báo rằng thị trường sẽ cần thêm thời gian để điều chỉnh và tìm lại điểm cân bằng mới. 
Trong dài hạn, với dân số trẻ và tốc độ đô thị hóa nhanh, tiềm năng phát triển của thị trường bất động sản Việt Nam vẫn được đánh giá cao. 
Các yếu tố như cải thiện cơ sở hạ tầng, sự minh bạch trong quy hoạch và chính sách pháp lý ổn định sẽ 

In [29]:
longer_vietnamese_text = """
Hà Nội mùa thu, cây cơm nguội vàng, cây bàng lá đỏ, nằm kề bên nhau, phố xưa nhà cổ, mái ngói thâm nâu. 
Hồ Gươm, Tháp Rùa, nghiêng soi bóng, nước xanh như lọc. 
Những con đường ngoại ô thành phố vẫn giữ được vẻ đẹp bình dị với những cánh đồng lúa xanh mướt và những đàn cò trắng bay lượn. 
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống. 
Mỗi độ thu về, Hà Nội lại khoác lên mình một chiếc áo mới, lãng mạn và đầy chất thơ, níu chân biết bao du khách.
"""

print("--- Summarization Comparison --- ")
print(f"Original Text (length: {len(longer_vietnamese_text)} chars):\n{longer_vietnamese_text}")

# TextRank
textrank_comp_summary = run_extractive_summarizer(longer_vietnamese_text, chosen_ratio=0.3)
print(f"\nTextRank Summary (ratio 0.3, length: {len(textrank_comp_summary)} chars):\n{textrank_comp_summary}")

# Abstractive (VietAI T5)
if model_summ and tokenizer_summ:
    abstractive_comp_summary = abstractive_summarize_vietnamese(longer_vietnamese_text, model_summ, tokenizer_summ)
    print(f"\nAbstractive Summary (VietAI T5, length: {len(abstractive_comp_summary)} chars):\n{abstractive_comp_summary}")
else:
    print("\nAbstractive summarization model not loaded. Skipping comparison for it.")

--- Summarization Comparison --- 
Original Text (length: 528 chars):

Hà Nội mùa thu, cây cơm nguội vàng, cây bàng lá đỏ, nằm kề bên nhau, phố xưa nhà cổ, mái ngói thâm nâu. 
Hồ Gươm, Tháp Rùa, nghiêng soi bóng, nước xanh như lọc. 
Những con đường ngoại ô thành phố vẫn giữ được vẻ đẹp bình dị với những cánh đồng lúa xanh mướt và những đàn cò trắng bay lượn. 
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống. 
Mỗi độ thu về, Hà Nội lại khoác lên mình một chiếc áo mới, lãng mạn và đầy chất thơ, níu chân biết bao du khách.


TextRank Summary (ratio 0.3, length: 121 chars):
Người dân Hà Nội nổi tiếng với sự thanh lịch, mến khách và tài hoa trong ẩm thực cũng như các nghề thủ công truyền thống.

Abstractive Summary (VietAI T5, length: 372 chars):
Những ngày đầu đông lạnh giá, rong ruổi trên các tuyến phố Hà Nội, bạn sẽ thấy những chiếc áo mới, những cánh đồng lúa xanh mướ, hay những đàn cò rắng bay lượn trên lối đi nội ô

In [23]:
# (Assuming sample_vietnamese_article_for_abstractive is defined)
# ... (model and tokenizer loading cell should be run first) ...

print("--- Abstractive Summarization Demo (VietAI/vit5-base-vietnews-summarization) ---")
print(f"Original Article (length: {len(sample_vietnamese_article_for_abstractive)} chars):\n{sample_vietnamese_article_for_abstractive}")

if model_summ and tokenizer_summ:
    # Example 1: Using default lengths
    print("\n--- Summarizing with default lengths ---")
    abstractive_summary_default = abstractive_summarize_vietnamese(
        sample_vietnamese_article_for_abstractive, 
        model_summ, 
        tokenizer_summ
    )
    print(f"Abstractive Summary (default lengths, length: {len(abstractive_summary_default)} chars):\n{abstractive_summary_default}")

    # Example 2: Using a target ratio (e.g., 30% of input sentences)
    print("\n--- Summarizing with target_ratio=0.3 ---")
    desired_ratio = 0.3
    abstractive_summary_ratio = abstractive_summarize_vietnamese(
        sample_vietnamese_article_for_abstractive, 
        model_summ, 
        tokenizer_summ,
        target_ratio=desired_ratio 
    )
    print(f"Abstractive Summary (ratio ~{desired_ratio}, length: {len(abstractive_summary_ratio)} chars):\n{abstractive_summary_ratio}")

    # Example 3: Using a target ratio (e.g., 60% of input sentences)
    print("\n--- Summarizing with target_ratio=0.6 ---")
    desired_ratio_2 = 0.6
    abstractive_summary_ratio_2 = abstractive_summarize_vietnamese(
        sample_vietnamese_article_for_abstractive, 
        model_summ, 
        tokenizer_summ,
        target_ratio=desired_ratio_2
    )
    print(f"Abstractive Summary (ratio ~{desired_ratio_2}, length: {len(abstractive_summary_ratio_2)} chars):\n{abstractive_summary_ratio_2}")
else:
    print("\nAbstractive summarization model not loaded. Skipping demo.")


--- Abstractive Summarization Demo (VietAI/vit5-base-vietnews-summarization) ---
Original Article (length: 751 chars):

Đại dịch COVID-19 đã gây ra những tác động sâu rộng đến kinh tế toàn cầu trong suốt hơn hai năm qua. 
Nhiều ngành công nghiệp chủ chốt như du lịch, hàng không, và dịch vụ ăn uống đã phải đối mặt với những thách thức chưa từng có. 
Các chính phủ trên khắp thế giới đã tung ra các gói kích thích kinh tế khổng lồ để hỗ trợ doanh nghiệp và người lao động. 
Tuy nhiên, quá trình phục hồi được dự báo sẽ không đồng đều và còn nhiều bất ổn phía trước, đặc biệt là với sự xuất hiện của các biến chủng mới. 
Chuyển đổi số và ứng dụng công nghệ được coi là chìa khóa quan trọng giúp các nền kinh tế thích ứng và vượt qua khủng hoảng. 
Việc đảm bảo chuỗi cung ứng toàn cầu và tăng cường hợp tác quốc tế cũng là những yếu tố then chốt cho sự phục hồi bền vững.


--- Summarizing with default lengths ---
  Ratio not provided or invalid, using default summary lengths: min=100, max=500
Abstra